CNN With PyTorch CalTech Dataset: https://towardsdatascience.com/transfer-learning-with-convolutional-neural-networks-in-pytorch-dd09190245ce



BERT SMS Spam detection with PyTorch SMS Spam detection: https://www.analyticsvidhya.com/blog/2020/07/transfer-learning-for-nlp-fine-tuning-bert-for-text-classification/